## a. Import required libraries

In [1]:
import cv2
import glob
import random
import numpy as np

print("Versions of key libraries")
print("---")
print("cv2: ", cv2.__version__)
print("numpy: ", np.__version__)

Versions of key libraries
---
cv2:  3.4.1
numpy:  1.19.1


## 1. Pre-processing: Extracting faces and gray scale

In [2]:
faceDet = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml")

In [12]:
# Define emotions
emotions = ["Angry", "Happy", "Neutral", "Sad", "Surprise"] 

In [10]:
def detect_faces(emotion):
    # Get list of all images with emotion
    files = glob.glob("../Data/Dataset_Original/%s/*" %emotion) 
    filenumber = 0
    for file in files:
        # Open image and read in gray scale
        frame = cv2.imread(file, 0)
        
        # Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_two = faceDet_two.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_three = faceDet_three.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_four = faceDet_four.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        
        # Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            facefeatures = face
        elif len(face_two) == 1:
            facefeatures = face_two
        elif len(face_three) == 1:
            facefeatures = face_three
        elif len(face_four) == 1:
            facefeatures = face_four
        else:
            facefeatures = ""
        
        # Cut and save face
        for (x, y, w, h) in facefeatures:  
            print ("Face found in file: %s" %file)
            
            # Cut the frame to size
            frame = frame[y:y+h, x:x+w] 
            try:
                # Resize face so all images have same size
                out = cv2.resize(frame, (160, 160)) 
                
                # Write image
                wrote = cv2.imwrite("../Data/Dataset_Preprocessed/%s/%s.png" % (emotion, filenumber), out) 
                print (wrote)
            except:
                # If error, pass file
                print("Error in processing %s" %file)
                pass
        
        # Increment image number
        filenumber += 1 

In [11]:
# Call function        
for emotion in emotions:
    detect_faces(emotion) 

Face found in file: ../Data/Dataset_Original/Angry\KA.AN1.39.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KA.AN2.40.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KA.AN3.41.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KL.AN1.167.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KL.AN2.168.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KM.AN1.17.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KM.AN2.18.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KM.AN3.19.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KR.AN1.83.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KR.AN2.84.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\KR.AN3.85.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\MK.AN1.125.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\MK.AN2.126.tiff
True
Face found in file: ../Data/Dataset_Original/Angry\MK.AN3.127.tiff
True
F

## 2. Creating the training and classification set

In [17]:
# Initialize fisher face classifier
fishface = cv2.face.FisherFaceRecognizer_create()
data = {}

# Define function to get file list, randomly shuffle it and split 80/20
def get_files(emotion): 
    files = glob.glob("../Data/Dataset_Preprocessed/%s/*" %emotion)
    random.shuffle(files)
    
    # Get first 80% of file list
    training = files[:int(len(files)*0.8)] 
    
    # Get last 20% of file list
    prediction = files[-int(len(files)*0.2):] 
    
    return training, prediction

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        
        # Append data to training and prediction list, and generate labels 0-7
        for item in training:
            # Open image
            image = cv2.imread(item, 0) 
            
            # Append image array to training data list
            training_data.append(image) 
            training_labels.append(emotions.index(emotion))
            
        # Repeat above process for prediction set
        for item in prediction: 
            image = cv2.imread(item, 0)
            prediction_data.append(image)
            prediction_labels.append(emotions.index(emotion))
            
    return training_data, training_labels, prediction_data, prediction_labels

def run_recognizer(i):
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    print ("Training Fisher Face Classifier %s" %i)
    print ("Size of training set is:", len(training_labels), "images")
    fishface.train(training_data, np.asarray(training_labels))
    print ("Predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

## 3. Run the model and save it

In [18]:
metascore = []
for i in range(1,11):
    correct = run_recognizer(i)
    print ("Achieved", correct, "percent correct!")
    metascore.append(correct) 
print ("\n\nEnd score:", np.mean(metascore), "percent correct!")

Training Fisher Face Classifier 1
Size of training set is: 921 images
Predicting classification set
Achieved 87.77292576419214 percent correct!
Training Fisher Face Classifier 2
Size of training set is: 921 images
Predicting classification set
Achieved 82.53275109170306 percent correct!
Training Fisher Face Classifier 3
Size of training set is: 921 images
Predicting classification set
Achieved 81.22270742358079 percent correct!
Training Fisher Face Classifier 4
Size of training set is: 921 images
Predicting classification set
Achieved 82.09606986899563 percent correct!
Training Fisher Face Classifier 5
Size of training set is: 921 images
Predicting classification set
Achieved 78.60262008733625 percent correct!
Training Fisher Face Classifier 6
Size of training set is: 921 images
Predicting classification set
Achieved 76.85589519650655 percent correct!
Training Fisher Face Classifier 7
Size of training set is: 921 images
Predicting classification set
Achieved 86.02620087336244 percent c

In [19]:
fishface.save("../Data/Models/emotion_detection_model_160.xml")